In [1]:
import pandas as pd
import os
from pathlib import Path
import platform

#Directorio
directory = "HSConversionTables"
if platform.system()=='Windows':
    Path('C:/Users/Asus/Desktop/CEP/pedidos Igal/'+directory).mkdir(parents=True, exist_ok=True)
    os.chdir('C:/Users/Asus/Desktop/CEP/pedidos Igal'+directory)
else: 
    Path('/home/nachengue/Escritorio/CEP/'+directory).mkdir(parents=True, exist_ok=True)
    os.chdir('/home/nachengue/Escritorio/CEP/'+directory)
print(os.getcwd())

data = pd.read_excel("./data/CompleteCorrelationsOfHS-SITC-BEC_20170606.xlsx")
cols = ['HS92','HS96','HS02','HS07','HS12','HS17']
data = data[cols].dropna().drop_duplicates()
for col in cols: 
    data[col] = data[col].apply(lambda x: col+"-"+str(int(x)).zfill(6))
connections = []
for i in range(len(cols)-1): 
        temp_tup_list = list(data[[cols[i], cols[i+1]]].drop_duplicates().itertuples(index=False, name=None))
        connections = connections + temp_tup_list

/home/nachengue/Escritorio/CEP/HSConversionTables


In [3]:
import networkx as nx

G = nx.Graph()
G.add_edges_from(connections)

In [20]:
#len(list(nx.connected_components(G))) #cantidad de posiciones homogenizables: 3832 
#len([x for x in list(nx.connected_components(G)) if len(x)==6]) #cantidad de grupos con relacion 1:1: 3328
#len([x for x in list(nx.connected_components(G)) if len(x)>6]) #cantidad de grupos con más de 6 componentes: 504
#len(data.HS17.unique()) #cantidad de posiciones distintas en HS17: 5388
#sorted(nx.node_connected_component(G, "HS17-854233"))

5388

In [13]:
type(nx.node_connected_component(G, "HS17-010190"))

set

In [30]:
# from statistics import mean, median
# print(mean([len(x) for x in list(nx.connected_components(G))]))
# print()
# print(median([len(x) for x in list(nx.connected_components(G))]))

8.09159707724426

6.0


In [5]:
q = "HS17-854233"
positions = sorted(nx.node_connected_component(G, q))

In [11]:
import re

q = "HS17-010190"
positions = sorted(nx.node_connected_component(G, q))

for i in range(len(cols)): 
    r = re.compile("|".join(cols[i:]))
    filtered_list = list(filter(r.match, positions))
    if len(filtered_list)==len(cols[i:]):
        print(f"you'll have homogeneus serie with {cols[:i]}")
        break

In [7]:
for i in range(len(cols)): 
    print(cols[i:])

['HS92', 'HS96', 'HS02', 'HS07', 'HS12', 'HS17']
['HS96', 'HS02', 'HS07', 'HS12', 'HS17']
['HS02', 'HS07', 'HS12', 'HS17']
['HS07', 'HS12', 'HS17']
['HS12', 'HS17']
['HS17']
